In [1]:
import os

In [ ]:
print(os.environ.get('HF_HUB_CACHE'))
os.environ['HF_HUB_CACHE'] = '~/my_huggingface_cache'
print(os.environ.get('HF_HUB_CACHE'))

/cache/huggingface/hub
~/my_huggingface_cache


In [3]:
import transformers
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer
from transformers import GPT2LMHeadModel

2025-03-24 20:43:33.104657: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742874213.120684  393938 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742874213.125528  393938 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-24 20:43:33.144785: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained('nferruz/ProtGPT2', cache_dir="/home/shai/my_huggingface_cache")
model = GPT2LMHeadModel.from_pretrained('nferruz/ProtGPT2', cache_dir="/home/shai/my_huggingface_cache").to(device) 

In [ ]:
def protgpt_wrapper(samples):
    ppls = []
    for seq in samples:
        out = tokenizer(seq, return_tensors="pt")
        input_ids = out.input_ids.cuda()

        with torch.no_grad():
            outputs = model(input_ids, labels=input_ids)

        ppl = (outputs.loss * input_ids.shape[1]).item()
        ppls.append(ppl)
    
    ppls = np.array(ppls)
    return ppls

def extract_ll_distr(df, seq_label):
    sequences = df[seq_label]
    return -1 * protgpt_wrapper(sequences)

In [6]:
pretrain_fn = 'data/baseline_data/distribution/original_old_10_0.5_0_results_merge.csv'
df_pretrain = pd.read_csv(pretrain_fn)
pretrain_ll = extract_ll_distr(df_pretrain, 'seq')

In [ ]:
df_pretrain['loglikelihood'] = pretrain_ll
df_pretrain.to_csv('test.csv', index=False)

In [ ]:
drakes_fn = 'data/baseline_data/distribution/original_new_10_0.5_0_results_merge.csv'
drakes_ll = extract_ll_distr(drakes_fn, 'seq')